# Vector Search of Movie Plots with Milvus

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sujee/data-prep-kit-examples/blob/main/milvus/milvus_2_movie_search.ipynb)


This notebook will demonstrate how do 'semantic search' of movie plots.  For example we can query for movies like:

- "Where humans fight aliens"
- "Relationship drama between two good friends"

We will:

- 👉 Use this [movie data](https://huggingface.co/datasets/MongoDB/embedded_movies)
- 👉 index the plot text using embedding models
- 👉 Load the indexed data into [Milvus](https://milvus.io/) -  a popular vector database.  
- 👉 And run queries

References
- [Milvus quick start](https://milvus.io/docs/quickstart.md)

**This notebook is deisnged to run on local python environment and Google Colab environment 😄**

## Configuration

### Embedding models

See hugging face embedding models (sentence transformers) here : https://huggingface.co/models?library=sentence-transformers&sort=trending

Here are a select models for comparison.  Taken from leaderboard : https://huggingface.co/spaces/mteb/leaderboard

| model name                              | overall score | model params | model size | embedding length | url                                                            |
|-----------------------------------------|---------------|--------------|------------|------------------|----------------------------------------------------------------|
| intfloat/e5-mistral-7b-instruct         | 66.x          | 7.11 B       | 15 GB      | 4096             | https://huggingface.co/intfloat/e5-mistral-7b-instruct         |
| BAAI/bge-large-en-v1.5                  | 64.x          | 335 M        | 1.34 GB    | 1024             | https://huggingface.co/BAAI/bge-large-en-v1.5                  |
| BAAI/bge-small-en-v1.5                  | 62.x          | 33.5 M       | 133 MB     | 384              | https://huggingface.co/BAAI/bge-small-en-v1.5                  |
| sentence-transformers/all-mpnet-base-v2 | 57.8          |              | 438 MB     | 768              | https://huggingface.co/sentence-transformers/all-mpnet-base-v2 |
| sentence-transformers/all-MiniLM-L12-v2 | 56.x          |              | 134 MB     | 384              | https://huggingface.co/sentence-transformers/all-MiniLM-L12-v2 |
| sentence-transformers/all-MiniLM-L6-v2  | 56.x          |              | 91 MB      | 384              | https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2  |


In [1]:
class MyConfig:
    pass

MY_CONFIG = MyConfig()

## different embedding models to try

# MY_CONFIG.MODEL_NAME = "bge-large-en-v1.5"
# MY_CONFIG.EMBEDDING_LENGTH = 1024

MY_CONFIG.MODEL_NAME = "BAAI/bge-small-en-v1.5"
MY_CONFIG.EMBEDDING_LENGTH = 384

# MY_CONFIG.MODEL_NAME = "all-mpnet-base-v2 "
# MY_CONFIG.EMBEDDING_LENGTH = 768

# MY_CONFIG.MODEL_NAME = "all-MiniLM-L6-v2"
# MY_CONFIG.EMBEDDING_LENGTH = 384



## Determine Runtime

In [2]:
# are we running in Colab?
import os

if os.getenv("COLAB_RELEASE_TAG"):
   print("Running in Colab")
   MY_CONFIG.RUNNING_IN_COLAB = True
else:
   print("NOT running in Colab")
   MY_CONFIG.RUNNING_IN_COLAB = False

NOT running in Colab


## Install Dependencies (If required)

**A note for Google Colab Users**

After installing the dependenceis, if you get errors loading libraries, **restart runtime** and **run the notebook** again

In [3]:
if MY_CONFIG.RUNNING_IN_COLAB:
  !pip install pymilvus  'pymilvus[model]'  datasets  sentence-transformers

## Load Data

We will load movies data.  The movie data has the following fields.  

- plot: A brief summary of the movie's plot.
- title: The title of the movie.
- and many more ...

See the [dataset description](https://huggingface.co/datasets/MongoDB/embedded_movies) for full description


In [4]:
from datasets import load_dataset

dataset = load_dataset("MongoDB/embedded_movies")['train']

# convert the dataset to an array of dicts, we only wants movies with plots
movies = [row for row in dataset if row['plot']]
print (f'Loaded {len(movies)} movies')

# select a few attributes
movies = [{k : v for k, v in m.items() if k in ['title', 'plot']} for m in movies ]

README.md:   0%|          | 0.00/6.17k [00:00<?, ?B/s]

sample_mflix.embedded_movies.json:   0%|          | 0.00/42.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Loaded 1473 movies


In [5]:
import pandas as pd
import pprint
import random

pprint.pprint (random.sample(movies, 5))
movies_df = pd.DataFrame(movies)
movies_df


[{'plot': 'In 2018, a mysterious new weapon in the war against the machines, '
          'half-human and half-machine, comes to John Connor on the eve of a '
          'resistance attack on Skynet. But whose side is he on, and can he be '
          'trusted?',
  'title': 'Terminator Salvation'},
 {'plot': 'A new take on the Manson Family murders, with a keen focus on '
          'Charles Manson himself.',
  'title': 'Helter Skelter'},
 {'plot': 'When the President of India is kidnapped by Kashmiri terrorists in '
          'Locarno, Switzerland, an Indian secret agent is sent to find him '
          'and set him free. With the help of an Indian singer he will make '
          'the "Impossible" to complete the mission.',
  'title': 'Asambhav'},
 {'plot': "In 1984, the USSR's best submarine captain in their newest sub "
          'violates orders and heads for the USA. Is he trying to defect, or '
          'to start a war?',
  'title': 'The Hunt for Red October'},
 {'plot': 'An imprison

,plot,title
0,Young Pauline is left a lot of money when her ...,The Perils of Pauline
1,A penniless young man tries to save an heiress...,From Hand to Mouth
2,"Michael ""Beau"" Geste leaves England in disgrac...",Beau Geste
3,"Seeking revenge, an athletic young man joins t...",The Black Pirate
4,An irresponsible young millionaire changes his...,For Heaven's Sake
...,...,...
1468,"In the ironically named city of Paradise, a re...",Postal
1469,A group of suburban biker wannabes looking for...,Wild Hogs
1470,"Shakespeare's masterpiece ""Othello"" set in mod...",Omkara
1471,When a small Colorado town is overrun by the f...,Day of the Dead


## Setup Embedded Database

Milvus can be embedded and easy to use.

After we execute this code, you will see `milvus_demo.db` and `.milvus_demo.db.lock` file in the folder

In [6]:
from pymilvus import MilvusClient

client = MilvusClient("milvus_demo.db")

# Create A Collection



In [7]:
# if we already have a collection, clear it first
if client.has_collection(collection_name="movies"):
    client.drop_collection(collection_name="movies")

client.create_collection(
    collection_name="movies",
    dimension=MY_CONFIG.EMBEDDING_LENGTH
)


## Calculate Embeddings for Plots

In [8]:
import torch

# Set the default device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print ('Using device : ', device)

Using device :  cuda


In [9]:
from pymilvus import model
import random

# If connection to https://huggingface.co/ failed, uncomment the following path
# import os
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# embedding_fn = model.DefaultEmbeddingFunction()

## initialize the SentenceTransformerEmbeddingFunction
embedding_fn = model.dense.SentenceTransformerEmbeddingFunction(
    model_name=MY_CONFIG.MODEL_NAME,
    device=device
)

# calculate embeddings for plots
for i, m in enumerate (movies):
    m['id'] = i
    m['vector'] = embedding_fn ([m['plot']][0])
    # m['vector'] = embedding_fn.encode_documents ([m['plot']][0])

# print a sample
for m in random.sample (movies, 3):
    print ('id:', m['id'] )
    print ('title: ', m['title'])
    print ('plot : ', m['plot'])
    print ('vector dim :',  len(m["vector"]))
    print ('vector[:10] :', m["vector"][:10])
    print()

/home/sujee/apps/anaconda3/envs/data-prep-kit-11-workshop/lib/python3.11/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


id: 108
title:  The Getaway
plot :  A recently released ex-con and his loyal wife go on the run after a heist goes awry.
vector dim : 384
vector[:10] : [-0.08382397, 0.05230988, -0.014309435, -0.011326386, 0.10341536, 0.056453865, 0.09219162, -0.0613747, 0.024395863, -0.091724716]

id: 1418
title:  National Treasure: Book of Secrets
plot :  Benjamin Gates must follow a clue left in John Wilkes Booth's diary to prove his ancestor's innocence in the assassination of Abraham Lincoln.
vector dim : 384
vector[:10] : [-0.031828713, 0.069093674, -0.0278634, 0.01077982, -0.022104671, -0.020216575, 0.052249935, -0.007882549, -0.06997956, -0.06649173]

id: 1202
title:  44 Minutes: The North Hollywood Shoot-Out
plot :  After a failed bank robbery, two heavily armed men hold the Los Angeles Police Department at bay for 44 minutes.
vector dim : 384
vector[:10] : [-0.044815417, 0.0033549126, -0.06647203, -0.022393027, 0.053098995, 0.024502177, 0.05405336, -0.044095457, 0.053354908, -0.085571066]



## Insert data

In [10]:
res = client.insert(collection_name="movies", data=movies)

print('inserted # rows', res['insert_count'])
print('cost', res['cost'])

inserted # rows 1473
cost 0


## Perform Vector Search (the FUN part!)

Let's do a semantic search on plot lines

In [11]:
from pprint import pprint

## helper function to perform vector search
def  do_vector_search (query):
    # query_vectors = embedding_fn.encode_queries([query])
    query_vectors = embedding_fn([query])

    results = client.search(
        collection_name="movies",  # target collection
        data=query_vectors,  # query vectors
        limit=5,  # number of returned entities
        output_fields=["title", "plot"],  # specifies fields to be returned
    )
    return results
## ----


def  print_search_results (results):
    # pprint (results)
    print ('num results : ', len(results[0]))

    for i, r in enumerate (results[0]):
        #pprint(r, indent=4)
        print (i+1)
        print ('search score:', r['distance'])
        print ('tile:', r['entity']['title'])
        print ('plot:', r['entity']['plot'])
        print()

In [12]:
query = "Where humans fight aliens"

results = do_vector_search (query)
print_search_results (results)

num results :  5
1
search score: 0.8236284852027893
tile: Independence Day
plot: The aliens are coming and their goal is to invade and destroy Earth. Fighting superior technology, mankind's best weapon is the will to survive.

2
search score: 0.7872651219367981
tile: Starship Troopers
plot: Humans in a fascistic, militaristic future do battle with giant alien bugs in a fight for survival.

3
search score: 0.7458122968673706
tile: V: The Final Battle
plot: A small group of human resistance fighters fight a desperate guerilla war against the genocidal extra-terrestrials who dominate Earth.

4
search score: 0.7356771230697632
tile: Enemy Mine
plot: A soldier from Earth crash-lands on an alien world after sustaining battle damage. Eventually he encounters another survivor, but from the enemy species he was fighting; they band together ...

5
search score: 0.7283806800842285
tile: Battlefield Earth
plot: After enslavement & near extermination by an alien race in the year 3000, humanity begi

In [13]:
query = "Relationship drama between friends"

results = do_vector_search (query)
print_search_results (results)

num results :  5
1
search score: 0.7513976693153381
tile: Varalaaru
plot: Relationships become entangled in an emotional web.

2
search score: 0.6959848403930664
tile: Once a Thief
plot: A romantic and action packed story of three best friends, a group of high end art thieves, who come into trouble when a love-triangle forms between them.

3
search score: 0.6907370090484619
tile: Dark Blue World
plot: The friendship of two men becomes tested when they both fall for the same woman.

4
search score: 0.6907370090484619
tile: Dark Blue World
plot: The friendship of two men becomes tested when they both fall for the same woman.

5
search score: 0.6906609535217285
tile: Harsh Times
plot: A tough-minded drama about two friends in South Central Los Angeles and the violence that comes between them.

